<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5: Text Classification
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

## Import libraries

In [13]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 12.0 MB 6.6 MB/s eta 0:00:011
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm -->
/opt/anaconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [23]:
## Import Libraries
import numpy as np
import pandas as pd

import string
import spacy

import re

from collections import Counter

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# import warnings
# warnings.filterwarnings('ignore')

## Load data

Sample:

    __label__2 Stuning even for the non-gamer: This sound ...
    __label__2 The best soundtrack ever to anything.: I'm ...
    __label__2 Amazing!: This soundtrack is my favorite m ...
    __label__2 Excellent Soundtrack: I truly like this so ...
    __label__2 Remember, Pull Your Jaw Off The Floor Afte ...
    __label__2 an absolute masterpiece: I am quite sure a ...
    __label__1 Buyer beware: This is a self-published boo ...
    . . .
    
There are only two **labels**:
- `__label__1`
- `__label__2`

In [118]:
## Loading the data

trainDF = pd.read_fwf(
    filepath_or_buffer = 'corpus.txt',
    colspecs = [(9, 10),   # label: get only the numbers 1 or 2
                (11, 9000) # text: makes the it big enought to get to the end of the line
               ], 
    header = 0,
    names = ['label', 'text'],
    lineterminator = '\n'
)

# convert label from [1, 2] to [0, 1]
trainDF['label'] = trainDF['label'] - 1

## Inspect the data

In [119]:
# ANSWER
trainDF.head()

,label,text
0,1,The best soundtrack ever to anything.: I'm rea...
1,1,Amazing!: This soundtrack is my favorite music...
2,1,Excellent Soundtrack: I truly like this soundt...
3,1,"Remember, Pull Your Jaw Off The Floor After He..."
4,1,an absolute masterpiece: I am quite sure any o...


## Split the data into train and test

In [120]:
## ANSWER
X = trainDF['text']
y = trainDF['label']
## split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Feature Engineering

### Count Vectors as features

In [121]:
# create a count vectorizer object
count_vect = CountVectorizer(token_pattern = r'\w{1,}')

# Learn a vocabulary dictionary of all tokens in the raw documents
count_vect.fit(trainDF['text'])

# Transform documents to document-term matrix.
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

### TF-IDF Vectors as features
- Word level
- N-Gram level
- Character level

In [122]:
%%time
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer = 'word',
                             token_pattern = r'\w{1,}',
                             max_features = 5000)
print(tfidf_vect)

tfidf_vect.fit(trainDF['text'])
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf  = tfidf_vect.transform(X_test)

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')
CPU times: user 1.44 s, sys: 38.1 ms, total: 1.48 s
Wall time: 1.49 s


In [123]:
%%time
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer = 'word',
                                   token_pattern = r'\w{1,}',
                                   ngram_range = (2, 3),
                                   max_features = 5000)
print(tfidf_vect_ngram)

tfidf_vect_ngram.fit(trainDF['text'])
X_train_tfidf_ngram = tfidf_vect_ngram.transform(X_train)
X_test_tfidf_ngram  = tfidf_vect_ngram.transform(X_test)

TfidfVectorizer(max_features=5000, ngram_range=(2, 3), token_pattern='\\w{1,}')
CPU times: user 6.35 s, sys: 337 ms, total: 6.69 s
Wall time: 6.73 s


In [124]:
%%time
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer = 'char',
                                         token_pattern = r'\w{1,}',
                                         ngram_range = (2, 3),
                                         max_features = 5000)
print(tfidf_vect_ngram_chars)

tfidf_vect_ngram_chars.fit(trainDF['text'])
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_train)
X_test_tfidf_ngram_chars  = tfidf_vect_ngram_chars.transform(X_test)

TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(2, 3),
                token_pattern='\\w{1,}')


/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 10.5 s, sys: 235 ms, total: 10.8 s
Wall time: 10.9 s


### Text / NLP based features

Create some other features.

Char_Count = Number of Characters in Text

Word Count = Number of Words in Text

Word Density = Average Number of Char in Words

Punctuation Count = Number of Punctuation in Text

Title Word Count = Number of Words in Title

Uppercase Word Count = Number of Upperwords in Text

In [125]:
%%time
# ANSWER
def char_count(string):
    return len(string)

word_pattern = '\w+'
def word_list(string):
    return re.findall(word_pattern, string)
def word_count(string):
    return len(word_list(string))
def word_dens(string):
    conc = ''.join(word_list(string))
    return len(conc)/word_count(string)

punc_pattern = '[\W\S]'
def punc_count(string):
    return len(re.findall(punc_pattern, string))

title_pattern = '.*:'
text_pattern  = ':.+'
def title_string(string):
    return re.findall(title_pattern, string)[0]
def text_string(string):
    return re.findall(text_pattern, string)[0][2:]

upper_pattern = '[A-Z]\w*'
def upper_count(string):
    return len(re.findall(upper_pattern, string))

CPU times: user 17 µs, sys: 37 µs, total: 54 µs
Wall time: 58.2 µs


In [126]:
def char_text(string):
    return char_count(text_string(string))

def word_text(string):
    return word_count(text_string(string))

def word_title(string):
    return(word_count(title_string(string)))

def dens_text(string):
    return word_dens(text_string(string))

def punc_text(string):
    return punc_count(text_string(string))

def upp_text(string):
    return upper_count(text_string(string))

In [127]:
trainDF['char_count'] = trainDF['text'].apply(char_text)

trainDF['word_count'] = trainDF['text'].apply(word_text)

trainDF['word_dens']  = trainDF['text'].apply(dens_text)

trainDF['punc_count'] = trainDF['text'].apply(punc_text)

trainDF['title_count']= trainDF['text'].apply(word_title)

trainDF['Word_Count'] = trainDF['text'].apply(upp_text)

In [128]:
## load spaCy
nlp = spacy.load('en')

Part of Speech in **SpaCy**

    POS   DESCRIPTION               EXAMPLES
    ----- ------------------------- ---------------------------------------------
    ADJ   adjective                 big, old, green, incomprehensible, first
    ADP   adposition                in, to, during
    ADV   adverb                    very, tomorrow, down, where, there
    AUX   auxiliary                 is, has (done), will (do), should (do)
    CONJ  conjunction               and, or, but
    CCONJ coordinating conjunction  and, or, but
    DET   determiner                a, an, the
    INTJ  interjection              psst, ouch, bravo, hello
    NOUN  noun                      girl, cat, tree, air, beauty
    NUM   numeral                   1, 2017, one, seventy-seven, IV, MMXIV
    PART  particle                  's, not,
    PRON  pronoun                   I, you, he, she, myself, themselves, somebody
    PROPN proper noun               Mary, John, London, NATO, HBO
    PUNCT punctuation               ., (, ), ?
    SCONJ subordinating conjunction if, while, that
    SYM   symbol                    $, %, §, ©, +, −, ×, ÷, =, :), 😝
    VERB  verb                      run, runs, running, eat, ate, eating
    X     other                     sfpksdpsxmsa
    SPACE space
    
Find out number of Adjective, Adverb, Noun, Numeric, Pronoun, Proposition, Verb.

    Hint:
    1. Convert text to spacy document
    2. Use pos_
    3. Use Counter 

In [129]:
# ANSWER

pos_list = ['ADJ', 'ADV', 'NOUN', 'NUM', 'PRON', 'PROPN', 'VERB']

zero_column = [0]*trainDF.shape[0]

for pos in pos_list:
    trainDF[pos] = zero_column

for index, row in trainDF.iterrows():
    string = row['text']
    my_series = []
    s_nlp = nlp(string)
    for token in s_nlp:
        my_series = my_series+[token.pos_]
    my_series = pd.Series(my_series)
    my_values = my_series.value_counts()
    for name, count in my_values.iteritems():
        if name in pos_list:
            trainDF.loc[index, name] = trainDF.loc[index, name] + count

In [130]:
trainDF.head()

,label,text,char_count,word_count,word_dens,punc_count,title_count,Word_Count,ADJ,ADV,NOUN,NUM,PRON,PROPN,VERB
0,1,The best soundtrack ever to anything.: I'm rea...,470,96,3.833333,470,6,11,7,4,18,1,9,2,15
1,1,Amazing!: This soundtrack is my favorite music...,750,135,4.333333,750,1,26,12,8,23,1,11,12,13
2,1,Excellent Soundtrack: I truly like this soundt...,721,120,4.783333,721,76,56,8,4,15,4,5,39,7
3,1,"Remember, Pull Your Jaw Off The Floor After He...",425,80,4.112500,425,60,22,6,3,12,0,8,16,11
4,1,an absolute masterpiece: I am quite sure any o...,800,146,4.308219,800,3,20,17,14,19,2,10,10,14


In [51]:
# Initialise some columns for feature's counts

In [53]:
#cols = [
#    'char_count', 'word_count', 'word_density',
#    'punctuation_count', 'title_word_count',
#    'uppercase_word_count', 'adj_count',
#    'adv_count', 'noun_count', 'num_count',
#    'pron_count', 'propn_count', 'verb_count']

#trainDF[cols].sample(5)

KeyError: "['word_density', 'title_word_count', 'uppercase_word_count', 'punctuation_count'] not in index"

### Topic Models as features

In [131]:
%%time
# train a LDA Model
lda_model = LatentDirichletAllocation(n_components = 20, learning_method = 'online', max_iter = 20)

X_topics = lda_model.fit_transform(X_train_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

CPU times: user 2min 15s, sys: 2.26 s, total: 2min 17s
Wall time: 2min 19s


In [132]:
# view the topic models
n_top_words = 10
topic_summaries = []
print('Group Top Words')
print('-----', '-'*80)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words = ' '.join(topic_words)
    topic_summaries.append(top_words)
    print('  %3d %s' % (i, top_words))

Group Top Words
----- --------------------------------------------------------------------------------
    0 the is cd and s music album of her a
    1 the and a of i to this it is in
    2 teaching losing bizarre awkward saga jesus mel disgusting orleans lay
    3 paper business adult hardly understanding political japanese bottle era musiq
    4 e genius social blues recordings dickens freedom impression describes tales
    5 humans grand technique scientific barbie argument lion nickelodeon chicken comic
    6 specific divorce additional guide seek jim dracula primer gluten curie
    7 recipes diary joe sung overpriced funk sandra whitney foods museum
    8 24 typos dragon midi bowls holidays 2006 chemistry hatebreed sloppy
    9 these boots pair boot them are they comfortable warm hi
   10 im california dont 120 greek chain circuit irrelevant birds sorbo
   11 the i it to and a this for my is
   12 haiku spine crawford remotely napoleon noir intentions praising oops painfully
   13

## Modelling

In [133]:
## helper function

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return accuracy_score(predictions, y_test)

In [134]:
# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Count Vectors',
                                  'WordLevel TF-IDF',
                                  'N-Gram Vectors',
                                  'CharLevel Vectors'])

### Naive Bayes Classifier

In [135]:
%%time
# Naive Bayes on Count Vectors
accuracy1 = train_model(MultinomialNB(), X_train_count, y_train, X_test_count)
print('NB, Count Vectors    : %.4f\n' % accuracy1)

NB, Count Vectors    : 0.8505

CPU times: user 9.22 ms, sys: 5.63 ms, total: 14.9 ms
Wall time: 19.4 ms


In [136]:
%%time
# Naive Bayes on Word Level TF IDF Vectors
accuracy2 = train_model(MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf)
print('NB, WordLevel TF-IDF : %.4f\n' % accuracy2)

NB, WordLevel TF-IDF : 0.8485

CPU times: user 7.33 ms, sys: 6.45 ms, total: 13.8 ms
Wall time: 12.3 ms


In [137]:
%%time
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy3 = train_model(MultinomialNB(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('NB, N-Gram Vectors   : %.4f\n' % accuracy3)

NB, N-Gram Vectors   : 0.8330

CPU times: user 6.24 ms, sys: 3.04 ms, total: 9.28 ms
Wall time: 8.03 ms


In [138]:
%%time
# # Naive Bayes on Character Level TF IDF Vectors
accuracy4 = train_model(MultinomialNB(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('NB, CharLevel Vectors: %.4f\n' % accuracy4)

NB, CharLevel Vectors: 0.8145

CPU times: user 30.4 ms, sys: 35.9 ms, total: 66.3 ms
Wall time: 64.6 ms


In [139]:
results.loc['Naïve Bayes'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Linear Classifier

In [140]:
%%time
# Linear Classifier on Count Vectors
accuracy1 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 350), X_train_count, y_train, X_test_count)
print('LR, Count Vectors    : %.4f\n' % accuracy1)

LR, Count Vectors    : 0.8555

CPU times: user 7.15 s, sys: 427 ms, total: 7.58 s
Wall time: 2.5 s


In [141]:
%%time
# Linear Classifier on Word Level TF IDF Vectors
accuracy2 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf, y_train, X_test_tfidf)
print('LR, WordLevel TF-IDF : %.4f\n' % accuracy2)

LR, WordLevel TF-IDF : 0.8680

CPU times: user 492 ms, sys: 57.2 ms, total: 550 ms
Wall time: 251 ms


In [142]:
%%time
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy3 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('LR, N-Gram Vectors   : %.4f\n' % accuracy3)

LR, N-Gram Vectors   : 0.8405

CPU times: user 155 ms, sys: 8.56 ms, total: 164 ms
Wall time: 75.8 ms


In [143]:
%%time
# Linear Classifier on Character Level TF IDF Vectors
accuracy4 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('LR, CharLevel Vectors: %.4f\n' % accuracy4)

LR, CharLevel Vectors: 0.8320

CPU times: user 1.01 s, sys: 19.2 ms, total: 1.03 s
Wall time: 366 ms


In [144]:
results.loc['Logistic Regression'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Support Vector Machine

In [145]:
%%time
# Support Vector Machine on Count Vectors
accuracy1 = train_model(LinearSVC(), X_train_count, y_train, X_test_count)
print('SVM, Count Vectors    : %.4f\n' % accuracy1)

SVM, Count Vectors    : 0.8350

CPU times: user 698 ms, sys: 10.2 ms, total: 708 ms
Wall time: 720 ms


In [146]:
%%time
# Support Vector Machine on Word Level TF IDF Vectors
accuracy2 = train_model(LinearSVC(), X_train_tfidf, y_train, X_test_tfidf)
print('SVM, WordLevel TF-IDF : %.4f\n' % accuracy2)

SVM, WordLevel TF-IDF : 0.8630

CPU times: user 86.5 ms, sys: 3.89 ms, total: 90.4 ms
Wall time: 92.6 ms


In [147]:
%%time
# Support Vector Machine on Ngram Level TF IDF Vectors
accuracy3 = train_model(LinearSVC(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('SVM, N-Gram Vectors   : %.4f\n' % accuracy3)

SVM, N-Gram Vectors   : 0.8215

CPU times: user 59.6 ms, sys: 2.19 ms, total: 61.8 ms
Wall time: 60 ms


In [148]:
%%time
# Support Vector Machine on Character Level TF IDF Vectors
accuracy4 = train_model(LinearSVC(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('SVM, CharLevel Vectors: %.4f\n' % accuracy4)

SVM, CharLevel Vectors: 0.8425

CPU times: user 423 ms, sys: 25.9 ms, total: 449 ms
Wall time: 493 ms


In [149]:
results.loc['Support Vector Machine'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Bagging Models

In [150]:
%%time
# Bagging (Random Forest) on Count Vectors
accuracy1 = train_model(RandomForestClassifier(n_estimators = 100), X_train_count, y_train, X_test_count)
print('RF, Count Vectors    : %.4f\n' % accuracy1)

RF, Count Vectors    : 0.8360

CPU times: user 11.9 s, sys: 71.4 ms, total: 12 s
Wall time: 12.1 s


In [151]:
%%time
# Bagging (Random Forest) on Word Level TF IDF Vectors
accuracy2 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf, y_train, X_test_tfidf)
print('RF, WordLevel TF-IDF : %.4f\n' % accuracy2)

RF, WordLevel TF-IDF : 0.8325

CPU times: user 5.54 s, sys: 35.8 ms, total: 5.57 s
Wall time: 5.66 s


In [152]:
%%time
# Bagging (Random Forest) on Ngram Level TF IDF Vectors
accuracy3 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('RF, N-Gram Vectors   : %.4f\n' % accuracy3)

RF, N-Gram Vectors   : 0.7925

CPU times: user 5.47 s, sys: 35.9 ms, total: 5.51 s
Wall time: 5.58 s


In [153]:
%%time
# Bagging (Random Forest) on Character Level TF IDF Vectors
accuracy4 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('RF, CharLevel Vectors: %.4f\n' % accuracy4)

RF, CharLevel Vectors: 0.7940

CPU times: user 19.9 s, sys: 154 ms, total: 20.1 s
Wall time: 20.7 s


In [154]:
results.loc['Random Forest'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Boosting Models

In [155]:
%%time
# Gradient Boosting on Count Vectors
accuracy1 = train_model(GradientBoostingClassifier(), X_train_count, y_train, X_test_count)
print('GB, Count Vectors    : %.4f\n' % accuracy1)

GB, Count Vectors    : 0.7965

CPU times: user 23 s, sys: 325 ms, total: 23.3 s
Wall time: 32.4 s


In [156]:
%%time
# Gradient Boosting on Word Level TF IDF Vectors
accuracy2 = train_model(GradientBoostingClassifier(), X_train_tfidf, y_train, X_test_tfidf)
print('GB, WordLevel TF-IDF : %.4f\n' % accuracy2)

GB, WordLevel TF-IDF : 0.7960

CPU times: user 12.4 s, sys: 293 ms, total: 12.7 s
Wall time: 21.5 s


In [157]:
%%time
# Gradient Boosting on Ngram Level TF IDF Vectors
accuracy3 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('GB, N-Gram Vectors   : %.4f\n' % accuracy3)

GB, N-Gram Vectors   : 0.7455

CPU times: user 7.12 s, sys: 92.5 ms, total: 7.22 s
Wall time: 7.73 s


In [158]:
%%time
# Gradient Boosting on Character Level TF IDF Vectors
accuracy4 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('GB, CharLevel Vectors: %.4f\n' % accuracy4)

GB, CharLevel Vectors: 0.8105

CPU times: user 1min 47s, sys: 1.15 s, total: 1min 48s
Wall time: 1min 53s


In [159]:
results.loc['Gradient Boosting'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

In [160]:
results

,Count Vectors,WordLevel TF-IDF,N-Gram Vectors,CharLevel Vectors
Naïve Bayes,0.8505,0.8485,0.8330,0.8145
Logistic Regression,0.8555,0.8680,0.8405,0.8320
Support Vector Machine,0.8350,0.8630,0.8215,0.8425
Random Forest,0.8360,0.8325,0.7925,0.7940
Gradient Boosting,0.7965,0.7960,0.7455,0.8105




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



